In [1]:
with open('./function/requirements_import.py') as f:
    exec(f.read())
    
from datetime import datetime
import scipy.stats as stats
from scipy.stats import spearmanr

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.outliers_influence import variance_inflation_factor

from datetime import datetime, timedelta

from scipy import interpolate
from scipy.stats import pearsonr

In [2]:
file_path = './data/rainfall_test.csv'
rainfall = pd.read_csv(file_path)
rainfall = rainfall.iloc[:, 1:]
rainfall.columns = rainfall.columns.str.replace('rainfall_test.', '')
rainfall

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,class_interval
0,D,5,1,9,STN031,3,D,5,1,12,68,46,13,1,0,0,0,0,0,NaN
1,D,5,1,9,STN031,6,D,5,1,15,83,66,26,5,0,0,0,0,0,NaN
2,D,5,1,9,STN031,9,D,5,1,18,17,6,0,0,0,0,0,0,0,NaN
3,D,5,1,9,STN031,12,D,5,1,21,0,0,0,0,0,0,0,0,0,NaN
4,D,5,1,9,STN031,15,D,5,2,0,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121995,D,9,30,21,STN035,228,D,10,10,9,4,4,4,2,2,0,0,0,0,NaN
121996,D,9,30,21,STN035,231,D,10,10,12,8,6,6,5,0,0,0,0,0,NaN
121997,D,9,30,21,STN035,234,D,10,10,15,8,6,6,5,0,0,0,0,0,NaN
121998,D,9,30,21,STN035,237,D,10,10,18,10,8,8,8,6,2,0,0,0,NaN


In [3]:
### 중간에 관측이 빠진 날짜 확인
idx = rainfall[rainfall['dh'] == 3].index #18239
idx2 = rainfall[rainfall['dh'] == 240].index #18159

### 240으로 80번 예측되지 않은 날이 있음을 확인 -> dic으로 리스트 추출
dic = {}

for i, j in enumerate(idx):
    if i == 0:
        continue
    
    t = rainfall.iloc[idx[i-1]:j]
    
    if len(t) != 80:
        tail = t.iloc[-1, 5]
        s1 = set(list(range(3,tail+1,3))) - set(t['dh'].values)
        
        if len(s1) > 0:
            a = t.iloc[0][:5]
            dic[f'{a[0]}_{a[1]}_{a[2]}_{a[3]}_{a[4]}'] = s1
        
dic

{}

In [4]:
# 18239일 중 160일이 빠진 관측시간이 있음
len(dic.keys())

0

-------

# 전처리

## 파생변수

In [9]:
### 시간형
col = ['fc_year','fc_month','fc_day','fc_hour'] + ['ef_year','ef_month','ef_day','ef_hour']
temp = rainfall[col]

temp[['fc_year', 'ef_year']] = temp[['fc_year', 'ef_year']].replace({'D': 2024}) #임시값
temp['datetime'] = temp.apply(lambda row: datetime(int(row['ef_year']), int(row['ef_month']), int(row['ef_day']), int(row['ef_hour'])), axis=1)

temp['quarter'] = temp['datetime'].dt.quarter
temp['timestamp'] = temp['datetime'].apply(lambda x: x.timestamp())
temp['timestamp_log'] = np.log(temp['timestamp'])
temp['ef_day_y'] = temp['datetime'].dt.dayofyear


temp['datetime2'] = temp.apply(lambda row: datetime(int(row['fc_year']), int(row['fc_month']), int(row['fc_day']), int(row['fc_hour'])), axis=1)
temp['fc_day_y'] = temp['datetime2'].dt.dayofyear

### 변수 추가하기
df_t = pd.concat([rainfall, temp[['quarter', 'timestamp_log', 'ef_day_y', 'fc_day_y']]], axis=1)

### 장소 str -> label
df_t['stn4contest'] = df_t['stn4contest'].str[-2:].astype(int)

df_t

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v05,v06,v07,v08,v09,class_interval,quarter,timestamp_log,ef_day_y,fc_day_y
0,D,5,1,9,31,3,D,5,1,12,...,0,0,0,0,0,NaN,2,21.262425,122,122
1,D,5,1,9,31,6,D,5,1,15,...,0,0,0,0,0,NaN,2,21.262431,122,122
2,D,5,1,9,31,9,D,5,1,18,...,0,0,0,0,0,NaN,2,21.262438,122,122
3,D,5,1,9,31,12,D,5,1,21,...,0,0,0,0,0,NaN,2,21.262444,122,122
4,D,5,1,9,31,15,D,5,2,0,...,0,0,0,0,0,NaN,2,21.262450,123,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121995,D,9,30,21,35,228,D,10,10,9,...,2,0,0,0,0,NaN,4,21.270549,284,274
121996,D,9,30,21,35,231,D,10,10,12,...,0,0,0,0,0,NaN,4,21.270555,284,274
121997,D,9,30,21,35,234,D,10,10,15,...,0,0,0,0,0,NaN,4,21.270562,284,274
121998,D,9,30,21,35,237,D,10,10,18,...,6,2,0,0,0,NaN,4,21.270568,284,274


In [10]:
### 구간별 강수확률
col = [f'v0{i}' for i in range(1,10)]
temp = rainfall[col]
temp2 = temp.shift(1, axis=1) - temp

# merge
temp = pd.concat([temp, temp2.iloc[:, 1:]], axis=1)
temp = temp.iloc[:, 8:]
temp.columns = [9] + list(range(1,8+1))

### 변수 추가하기
temp2 = temp[list(range(1,9+1))]
temp2.columns = [f'rng_v0{i}' for i in range(1,10)]
df_t = pd.concat([df_t, temp2], axis=1)


In [12]:
### pred
def max_find(row):
    # 행의 합이 0이면 0 반환
    if row.sum() == 0:
        return 0
    
    # 최대값 찾기
    max_value = max(row)

    # 최대값의 인덱스 찾기
    max_indices = [index for index, value in enumerate(row) if value == max_value]
    a = max(max_indices)

    return a

### 합치기
df_t['pred'] = temp.apply(max_find, axis=1)

df_t

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,rng_v01,rng_v02,rng_v03,rng_v04,rng_v05,rng_v06,rng_v07,rng_v08,rng_v09,pred
0,D,5,1,9,31,3,D,5,1,12,...,22,33,12,1,0,0,0,0,0,2
1,D,5,1,9,31,6,D,5,1,15,...,17,40,21,5,0,0,0,0,0,2
2,D,5,1,9,31,9,D,5,1,18,...,11,6,0,0,0,0,0,0,0,1
3,D,5,1,9,31,12,D,5,1,21,...,0,0,0,0,0,0,0,0,0,0
4,D,5,1,9,31,15,D,5,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121995,D,9,30,21,35,228,D,10,10,9,...,0,0,2,0,2,0,0,0,0,5
121996,D,9,30,21,35,231,D,10,10,12,...,2,0,1,5,0,0,0,0,0,4
121997,D,9,30,21,35,234,D,10,10,15,...,2,0,1,5,0,0,0,0,0,4
121998,D,9,30,21,35,237,D,10,10,18,...,2,0,0,2,4,2,0,0,0,5


In [15]:
### shift
col = [f'v0{i}' for i in range(2,6)]
val2 = [3,3,2,2]

for key, val in zip(col, val2):
    df_t[f'roll_{key}'] = df_t[[key]].rolling(val, min_periods=1).mean()

In [16]:
df_t.columns

Index(['fc_year', 'fc_month', 'fc_day', 'fc_hour', 'stn4contest', 'dh',
       'ef_year', 'ef_month', 'ef_day', 'ef_hour', 'v01', 'v02', 'v03', 'v04',
       'v05', 'v06', 'v07', 'v08', 'v09', 'class_interval', 'quarter',
       'timestamp_log', 'ef_day_y', 'fc_day_y', 'rng_v01', 'rng_v02',
       'rng_v03', 'rng_v04', 'rng_v05', 'rng_v06', 'rng_v07', 'rng_v08',
       'rng_v09', 'pred', 'roll_v02', 'roll_v03', 'roll_v04', 'roll_v05'],
      dtype='object')

In [17]:
temp = rainfall.loc[:, ['fc_month', 'stn4contest'] + [f'v0{i}' for i in range(1,10)]]

col = [f'v0{i}' for i in range(1,10)]
col1 = ['fc_month', 'stn4contest']
col2 = ['m', 'aws']

for a,b in zip(col1, col2):
    for val in col:
        dic = temp.groupby(a)[val].mean().to_dict()
        df_t[f'{b}_{val}'] = temp[a].map(dic)

In [27]:
df_t.to_csv(file_path + 'rainfall_test_v4_eda2.csv', index=False)

In [25]:
col = joblib.load(file_path + 'lis_col.pkl')
col.remove('pred_YN')
df_t[col]

,rng_v02,rng_v03,fc_day,m_v09,rng_v05,v05,aws_v03,v06,m_v08,m_v01,...,aws_v05,aws_v08,v02,v01,fc_day_y,aws_v09,ef_hour,timestamp_log,ef_day,pred
0,33,12,1,0.023156,0,0,22.626885,0,0.168197,18.515574,...,10.781230,0.470205,46,68,122,0.170779,12,21.262425,1,2
1,40,21,1,0.023156,0,0,22.626885,0,0.168197,18.515574,...,10.781230,0.470205,66,83,122,0.170779,15,21.262431,1,2
2,6,0,1,0.023156,0,0,22.626885,0,0.168197,18.515574,...,10.781230,0.470205,6,17,122,0.170779,18,21.262438,1,1
3,0,0,1,0.023156,0,0,22.626885,0,0.168197,18.515574,...,10.781230,0.470205,0,0,122,0.170779,21,21.262444,1,0
4,0,0,1,0.023156,0,0,22.626885,0,0.168197,18.515574,...,10.781230,0.470205,0,0,122,0.170779,0,21.262450,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121995,0,2,30,0.094875,2,2,20.946885,0,0.282042,25.422708,...,11.861148,0.579098,4,4,274,0.188074,9,21.270549,10,5
121996,0,1,30,0.094875,0,0,20.946885,0,0.282042,25.422708,...,11.861148,0.579098,6,8,274,0.188074,12,21.270555,10,4
121997,0,1,30,0.094875,0,0,20.946885,0,0.282042,25.422708,...,11.861148,0.579098,6,8,274,0.188074,15,21.270562,10,4
121998,0,0,30,0.094875,4,6,20.946885,2,0.282042,25.422708,...,11.861148,0.579098,8,10,274,0.188074,18,21.270568,10,5


# 최종

In [80]:
# df_t = df_t.drop(columns=['fc_year', 'ef_year'])
df_t.to_csv('./data/rainfall_train_v2.csv', index=False)

In [79]:
df_t.columns

Index(['fc_year', 'fc_month', 'fc_day', 'fc_hour', 'stn4contest', 'dh',
       'ef_year', 'ef_month', 'ef_day', 'ef_hour', 'v06', 'v07', 'v08', 'v09',
       'vv', 'class_interval', 'quarter', 'timestamp', 'ef_day_y', 'fc_day_y',
       'rng_v08', 'rng_v09', 'pred', 'pred_YN', 'roll_v01', 'roll_v02',
       'roll_v03', 'roll_v04', 'roll_v05', 'roll_rng_v01', 'roll_rng_v02',
       'roll_rng_v03', 'roll_rng_v04', 'roll_rng_v05', 'roll_rng_v06',
       'roll_rng_v07'],
      dtype='object')